In [14]:
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from skimage.feature import hog
from sklearn.decomposition import PCA
from sklearn.svm import SVC

%matplotlib inline

In [15]:
with open('.\dataset\ck.pikle','rb') as f:
    ck_data, ck_label = pickle.load(f)

In [33]:
hog = cv2.HOGDescriptor()
pca = PCA(n_components=327, svd_solver='randomized', whiten=True)
feature_hog_ck_data = np.array([])

first = True

for image in ck_data :
    image = cv2.resize(image, (128,128))
    feature = hog.compute(image, winStride=(4,4))
    feature = feature.T
    if first :
        feature_hog_ck_data = np.vstack(feature)
        first = False
    else :
        feature_hog_ck_data = np.vstack((feature_hog_ck_data, feature))
    
pca.fit(feature_hog_ck_data)
feature_hog_ck_data_pca = pca.transform(feature_hog_ck_data)
    
ck_data_train, ck_data_test, ck_label_train, ck_label_test = train_test_split(feature_hog_ck_data_pca, ck_label, test_size=0.25, random_state=42)

In [34]:
target_classes = ["Neutral", "Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise"]

clf = SVC(kernel='rbf', class_weight='balanced', gamma=0.0001, C=1000.0)

clf = clf.fit(ck_data_train, ck_label_train)
ck_label_predict = clf.predict(ck_data_test)

print(classification_report(ck_label_test, ck_label_predict, target_names=target_classes))
print(confusion_matrix(ck_label_test, ck_label_predict, labels=range(7)))

              precision    recall  f1-score   support

     Neutral       0.13      0.23      0.17        13
       Angry       0.56      0.36      0.43        14
     Disgust       0.67      0.35      0.46        17
        Fear       0.17      0.29      0.21         7
       Happy       0.18      0.18      0.18        11
         Sad       0.20      0.17      0.18         6
    Surprise       0.46      0.43      0.44        14

   micro avg       0.30      0.30      0.30        82
   macro avg       0.34      0.29      0.30        82
weighted avg       0.39      0.30      0.33        82

[[3 2 1 1 2 2 2]
 [3 5 0 2 2 1 1]
 [6 1 6 1 1 1 1]
 [2 0 0 2 1 0 2]
 [3 0 1 4 2 0 1]
 [2 1 1 1 0 1 0]
 [4 0 0 1 3 0 6]]
